In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from typing import List
import numpy as np
import joblib
import imodelsx.util
import sasc.viz
import pickle as pkl
from copy import deepcopy
from numpy.linalg import norm
from sasc.config import CACHE_DIR, RESULTS_DIR
import sasc.modules.fmri_module
cache_ngrams_dir = '/home/chansingh/mntv1/deep-fMRI/sasc/mprompt/cache/cache_ngrams'
regions_idxs_dir = '/home/chansingh/mntv1/deep-fMRI/sasc/brain_regions'

In [ ]:
ngrams_list = joblib.load(join(cache_ngrams_dir, 'fmri_UTS02_ngrams.pkl'))
rois_dict = joblib.load(join(regions_idxs_dir, 'rois_S02.jbl'))

In [ ]:
rois = ['RSC', 'OPA', 'PPA']
for k in rois:
    assert k in rois_dict

In [ ]:
mod = sasc.modules.fmri_module.fMRIModule(
    subject="UTS02",
    checkpoint="facebook/opt-30b",
    init_model=False,
    restrict_weights=False,
)
embs = joblib.load(join(cache_ngrams_dir, 'fmri_embs.pkl'))
outputs = mod(embs=embs, return_all=True)

In [ ]:
rois_dict[k]

In [21]:
outputs_dict = {
    k: outputs[:, np.array(rois_dict[k])].mean(axis=1)
    for k in rois
}
outputs.shape

(243, 94251)

In [27]:
pd.DataFrame(outputs_dict, index=ngrams_list[:len(outputs)])

,RSC,OPA,PPA
!,0.011897,-0.013787,-0.000993
! could,0.018953,-0.013941,0.010274
! could like,0.013322,-0.004487,0.008942
',0.004320,-0.031278,-0.007667
' 's,0.031145,0.004670,0.013857
...,...,...,...
'd just say,0.019364,-0.006650,0.002816
'd just started,0.026087,-0.003321,0.009467
'd learnt,0.006257,-0.023387,-0.002731
'd learnt to,0.038807,-0.003242,0.012137
